# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921699782622                   -0.69    6.0         
  2   -7.926164701553       -2.35       -1.22    1.0    289ms
  3   -7.926837697714       -3.17       -2.37    1.8    324ms
  4   -7.926861498111       -4.62       -3.04    2.6    389ms
  5   -7.926861647000       -6.83       -3.42    2.0    306ms
  6   -7.926861670212       -7.63       -3.81    1.6    292ms
  7   -7.926861680313       -8.00       -4.31    1.1    274ms
  8   -7.926861681774       -8.84       -4.88    1.9    341ms
  9   -7.926861681859      -10.07       -5.25    1.9    303ms
 10   -7.926861681871      -10.94       -5.81    1.8    301ms
 11   -7.926861681873      -11.80       -6.74    1.8    335ms
 12   -7.926861681873      -13.26       -7.61    2.6    351ms
 13   -7.926861681873   +  -15.05       -7.93    2.8    334ms
 14   -7.926861681873      -15.05       -8.67    2.0    315ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921695943117                   -0.69    5.9         
  2   -7.926163477485       -2.35       -1.22    1.0    285ms
  3   -7.926837481789       -3.17       -2.37    1.9    398ms
  4   -7.926861528110       -4.62       -3.03    2.6    337ms
  5   -7.926861643076       -6.94       -3.40    1.6    288ms
  6   -7.926861669837       -7.57       -3.80    1.6    336ms
  7   -7.926861680466       -7.97       -4.35    1.4    291ms
  8   -7.926861681786       -8.88       -4.89    2.0    303ms
  9   -7.926861681863      -10.12       -5.33    1.9    308ms
 10   -7.926861681872      -11.04       -5.92    1.9    334ms
 11   -7.926861681873      -12.00       -7.29    1.9    315ms
 12   -7.926861681873      -13.56       -7.51    3.8    389ms
 13   -7.926861681873   +  -14.57       -8.66    1.1    282ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921697620034                   -0.69    5.8         
  2   -7.926170463353       -2.35       -1.22    1.0    272ms
  3   -7.926839233929       -3.17       -2.37    1.8    310ms
  4   -7.926864900950       -4.59       -3.01    2.8    339ms
  5   -7.926865044022       -6.84       -3.34    1.9    355ms
  6   -7.926865076905       -7.48       -3.71    1.4    263ms
  7   -7.926865091537       -7.83       -4.35    1.2    257ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
